In [3]:
import concurrent.futures
import requests# This is not standard library

MAX_WORKERS = 20#要使用最大的執行緒數量
URLS = [
    'https://docs.python.org/3/library/ast.html',
    'https://docs.python.org/3/library/abc.html',
    'https://docs.python.org/3/library/time.html',
    'https://docs.python.org/3/library/os.html',
    'https://docs.python.org/3/library/sys.html',
    'https://docs.python.org/3/library/io.html',
    'https://docs.python.org/3/library/pdb.html',
    'https://docs.python.org/3/library/weakref.html'
]

def get_content(url):
    return requests.get(url).text

#一個一個慢慢抓
def main():
    for url in URLS:
        try:
            data = get_content(url)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page length is %d' % (url, len(data)))

#並行抓
def scrap():
    workers = min(MAX_WORKERS, len(URLS))#避免建立不必要的執行緒
    with concurrent.futures.ThreadPoolExecutor(workers) as executor:
        future_to_url = {executor.submit(get_content, url): url for url in URLS}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Execption as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page length is %d' % (url, len(data)))

if __name__ == '__main__':
    scrap()

'https://docs.python.org/3/library/pdb.html' page length is 57467
'https://docs.python.org/3/library/weakref.html' page length is 64892
'https://docs.python.org/3/library/time.html' page length is 93830
'https://docs.python.org/3/library/abc.html' page length is 41779
'https://docs.python.org/3/library/os.html' page length is 506340
'https://docs.python.org/3/library/io.html' page length is 129875
'https://docs.python.org/3/library/ast.html' page length is 231035
'https://docs.python.org/3/library/sys.html' page length is 177059


In [8]:
import concurrent.futures
import requests# This is not standard library

MAX_WORKERS = 20
URLS = [
    'https://docs.python.org/3/library/ast.html',
    'https://docs.python.org/3/library/abc.html',
    'https://docs.python.org/3/library/time.html',
    'https://docs.python.org/3/library/os.html',
    'https://docs.python.org/3/library/sys.html',
    'https://docs.python.org/3/library/io.html',
    'https://docs.python.org/3/library/pdb.html',
    'https://docs.python.org/3/library/weakref.html'
]

def get_content(url):
    return requests.get(url).text

#一個一個慢慢抓
def main():
    for url in URLS:
        try:
            data = get_content(url)
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page length is %d' % (url, len(data)))

#並行抓
def scrap():
    workers = min(MAX_WORKERS, len(URLS))
    with concurrent.futures.ThreadPoolExecutor(workers) as executor:
        future_to_url = {executor.submit(get_content, url): url for url in URLS}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Execption as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page length is %d' % (url, len(data)))

if __name__ == '__main__':
    %time main()
    %time scrap()

'https://docs.python.org/3/library/ast.html' page length is 231035
'https://docs.python.org/3/library/abc.html' page length is 41779
'https://docs.python.org/3/library/time.html' page length is 93830
'https://docs.python.org/3/library/os.html' page length is 506340
'https://docs.python.org/3/library/sys.html' page length is 177059
'https://docs.python.org/3/library/io.html' page length is 129875
'https://docs.python.org/3/library/pdb.html' page length is 57467
'https://docs.python.org/3/library/weakref.html' page length is 64892
Wall time: 1.72 s
'https://docs.python.org/3/library/abc.html' page length is 41779
'https://docs.python.org/3/library/time.html' page length is 93830
'https://docs.python.org/3/library/ast.html' page length is 231035
'https://docs.python.org/3/library/pdb.html' page length is 57467
'https://docs.python.org/3/library/weakref.html' page length is 64892
'https://docs.python.org/3/library/sys.html' page length is 177059
'https://docs.python.org/3/library/io.html' 

In [5]:
#換一種寫法，作為觀察用
#ThreadPoolExecutor適合IO密集
#ProcessPoolExecutor適合CPU密集

import concurrent.futures
import requests# This is not standard library

MAX_WORKERS = 20
URLS = [
    'https://docs.python.org/3/library/ast.html',
    'https://docs.python.org/3/library/abc.html',
    'https://docs.python.org/3/library/time.html',
    'https://docs.python.org/3/library/os.html',
    'https://docs.python.org/3/library/sys.html',
    'https://docs.python.org/3/library/io.html',
    'https://docs.python.org/3/library/pdb.html',
    'https://docs.python.org/3/library/weakref.html'
]

def get_content(url):
    return requests.get(url).text

#並行抓
def scrap():
    #workers = min(MAX_WORKERS, len(URLS))
    workers=3 #設為3 可觀察輸出的執行緒有3個running，其餘pending
    with concurrent.futures.ThreadPoolExecutor(workers) as executor:#這⾥我們創建了⼀個線程池
        
        future_to_url={}
        
        for url in URLS:
            future = executor.submit(get_content, url)
            future_to_url[ future ]=url #安排可呼叫執行的程序
            print('Scheduled for %r: %s' % (url,future))
        for future in concurrent.futures.as_completed(future_to_url): #as_completed會在完成後產生future
            url = future_to_url[future]
            try:
                data = future.result() #取得資料，future.result永遠不會阻塞
            except Execption as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page length is %d' % (url, len(data)))

if __name__ == '__main__':
    scrap()

Scheduled for 'https://docs.python.org/3/library/ast.html': <Future at 0x24815ac3a88 state=running>
Scheduled for 'https://docs.python.org/3/library/abc.html': <Future at 0x248161b5e48 state=running>
Scheduled for 'https://docs.python.org/3/library/time.html': <Future at 0x248161f4248 state=running>
Scheduled for 'https://docs.python.org/3/library/os.html': <Future at 0x24813772b88 state=pending>
Scheduled for 'https://docs.python.org/3/library/sys.html': <Future at 0x24815cf0688 state=pending>
Scheduled for 'https://docs.python.org/3/library/io.html': <Future at 0x2481530aa48 state=pending>
Scheduled for 'https://docs.python.org/3/library/pdb.html': <Future at 0x2481530aec8 state=pending>
Scheduled for 'https://docs.python.org/3/library/weakref.html': <Future at 0x2481530a808 state=pending>
'https://docs.python.org/3/library/abc.html' page length is 41779
'https://docs.python.org/3/library/time.html' page length is 93830
'https://docs.python.org/3/library/ast.html' page length is 2310

In [ ]:
#execute.submit與future.as_completed的組合比executor.map還要靈活
#executor.map的設計是為了用不同引數來執行相同的可呼叫物
#而future.as_completed接收的future集合可能來自一個以上的executor